# 주간 Melon Top100 크롤링

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import random

# 크롤링
* 2022년 1 ~ 3월

In [18]:
# 수집할 월 지정
end_month = 3

## 멜론 사이트 접속

In [19]:
driver = webdriver.Chrome('../chromedriver')
url = 'https://www.melon.com/chart/index.htm'
driver.get(url)
driver.maximize_window()

<ipython-input-19-40ce6c51e4f5>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver')


In [20]:
# 차트 파인더로 이동
chartfinder_button = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button')
chartfinder_button.click()
time.sleep( random.uniform(1, 2) )

# 주간차트 선택
weekchart_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a')
weekchart_button.click()
time.sleep( random.uniform(1, 2) )

# 연대선택(2020년대)
selecttopyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label')
selecttopyear_button.click()
time.sleep( random.uniform(1, 2) )

# 연도선택(2022년)
selectyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label')
selectyear_button.click()
time.sleep( random.uniform(1, 2) )

# 월간 선택(1월)
selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label')
selectmonth.click()
time.sleep( random.uniform(1, 2) )

# 주간 선택(1주)
selectweek_button = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[1]/span/label')
selectweek_button.click()
time.sleep( random.uniform(1, 2) )

# 장르 선택
selectgenre_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label')
selectgenre_button.click()
time.sleep( random.uniform(1, 2) )

# 검색 버튼 클릭
search_button = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span')
search_button.click()
time.sleep( random.uniform(1, 2) )

# 처음 검색 한번 한뒤에 진행해야 정상적으로 첫번째 정보 정상적으로 수집.
# 바로 진행하면 2013년 자료 수집이 되는 문제 발생.

<ipython-input-20-8e63b148ae1f>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  chartfinder_button = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button')
<ipython-input-20-8e63b148ae1f>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  weekchart_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a')
<ipython-input-20-8e63b148ae1f>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selecttopyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label')
<ipython-input-20-8e63b148ae1f>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectyear_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/di

In [21]:
for m in range(1, end_month + 1):
    # 월간 선택
    selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{m}]/span/label')
    selectmonth.click()
    time.sleep( random.uniform(1, 2) )

    len_week = driver.find_elements_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li')
    num = len(len_week) + 1
    # 주간 선택
    for w in range(1, num):
        selectweek_button = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[{w}]/span/label')
        selectweek_button.click()
        time.sleep( random.uniform(1, 2) )

        # 장르 선택
        selectgenre_button = driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label')
        selectgenre_button.click()
        time.sleep( random.uniform(1, 2) )

        # 검색 버튼 클릭
        search_button = driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span')
        search_button.click()
        time.sleep( random.uniform(1, 2) )
        
        # 정보 수프에 담기
        soup = bs(driver.page_source, 'lxml')

        # 검색 제목 수집
        search_title = soup.find('div', 'serch_cnt')
        title = search_title.text[:-8]

        # 순위 날짜
        search_title = soup.find('div', 'serch_cnt')
        temp = search_title.text[:10]
        temp = temp.replace('.', '-')
        repeat_rank_date_list = []
        for i in range(100):
            repeat_rank_date_list.append(temp)

        # 순위 수집
        rank_list = []
        ranks = soup.find_all('span', 'rank')
        for one in ranks:
            temp = one.text
            rank_list.append(temp)
        
        # 곡정보
        song_list = []
        song_info = soup.find_all('div', 'wrap_song_info')
        for one in song_info:
            temp = one.find('div', 'ellipsis rank01').text.strip()
            if '19금\n' in temp:
                temp = temp.replace('19금\n', '')
            song_list.append(temp)

        # 가수
        artist_list = []
        for one in song_info:
            temp = one.find('span', 'checkEllipsis').text
            artist_list.append(temp)
        # 앨범
        album_list = []
        for one in song_info:
            temp = one.find('div', 'ellipsis rank03').text
            album_list.append(temp)

        # 수집 데이터 확인
        print('수집한 기간 : ', title)
        print('수집한 순위 날짜 개수 : ', len(repeat_rank_date_list))
        print('수집한 순위 개수 : ', len(rank_list))
        print('수집한 곡 개수 : ', len(song_list))
        print('수집한 가수 개수 : ', len(artist_list))
        print('수집한 앨범 개수 : ', len(album_list))

        # 데이터 셋 만들기
        dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':song_list, '가수':artist_list, '앨범':album_list}
        df = pd.DataFrame(dict)

        # 엑셀 출력
        date = repeat_rank_date_list[0]
        date = date.replace('-', '')
        path = '../crawling_data/week_melon/'
        file_name = f'week_melon_{date}.xlsx'
        df.to_excel(path + file_name, index=False)
        print(f'{file_name} 출력 성공.')

<ipython-input-21-1722d35e010f>:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectmonth = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{m}]/span/label')
<ipython-input-21-1722d35e010f>:7: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  len_week = driver.find_elements_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li')
<ipython-input-21-1722d35e010f>:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectweek_button = driver.find_element_by_xpath(f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[{w}]/span/label')
<ipython-input-21-1722d35e010f>:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  selectgenre_button = driver.find_element_by_xpath

수집한 기간 :  2022.01.03 ~ 2022.01.09
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220103.xlsx 출력 성공.
수집한 기간 :  2022.01.10 ~ 2022.01.16
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220110.xlsx 출력 성공.
수집한 기간 :  2022.01.17 ~ 2022.01.23
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220117.xlsx 출력 성공.
수집한 기간 :  2022.01.24 ~ 2022.01.30
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220124.xlsx 출력 성공.
수집한 기간 :  2022.01.31 ~ 2022.02.06
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220131.xlsx 출력 성공.
수집한 기간 :  2022.02.07 ~ 2022.02.13
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 :  100
수집한 가수 개수 :  100
수집한 앨범 개수 :  100
week_melon_20220207.xlsx 출력 성공.
수집한 기간 :  2022.02.14 ~ 2022.02.20
수집한 순위 날짜 개수 :  100
수집한 순위 개수 :  100
수집한 곡 개수 : 

# 코드 마지막